Advanced Feature Selection using Feature Importance Rank Ensembling (FIRE)

<b>Authors:</b> Vitalii Peretiatko, Rajiv Shah

<b>Scope:</b><br>
This notebook shows the benefits of advanced feature selection that uses median rank agggregation of feature impacts across several models created during a run of DataRobot autopilot. 

<b>Background:</b>
 - Original blogpost: https://www.datarobot.com/blog/using-feature-importance-rank-ensembling-fire-for-advanced-feature-selection/ 
 - Blogpost on advanced feature selection in R: https://community.datarobot.com/t5/resources/advanced-feature-selection-with-r/ta-p/5307
 - Instructions and logic of how to do advanced feature selection in Python: https://github.com/datarobot-community/examples-for-data-scientists/blob/master/Feature%20Lists%20Manipulation/Python/Advanced%20Feature%20Selection.ipynb 
 
<b> Requirements </b><br>
Python version >= 3.7.3<br>
DataRobot API version >= 2.22.1.<br>
Full documentation of the Python package can be found here: https://datarobot-public-api-client.readthedocs-hosted.com

<b>Dataset</b><br>
We will be using Madelon dataset from this paper https://archive.ics.uci.edu/ml/datasets/Madelon <br>
It can also be found here https://s3.amazonaws.com/datarobot_public_datasets/madelon_combined_80.csv


<b>Import libraries and connect to DataRobot</b>

In [1]:
import numpy as np
import pandas as pd
import datarobot as dr

#connect to DataRobot using the config file
dr.Client(config_path='../../drconfig.yaml')

<b>Define the function for FIRE feature selection</b>

In [2]:
def feature_importance_rank_ensembling(project, 
                                          n_models=5, 
                                          metric=None, 
                                          by_partition='validation',
                                          feature_list_name=None,
                                          ratio=0.95,
                                          model_search_params=None, 
                                          use_ranks=True,
                                         ):   
    """
    Function that implements the logic of Feature Selection using Feature Importance Rank Ensembling and restarts DR autopilot
    
    Parameters:
    -----------
    project: DR project object, 
    n_models: int, get top N best models on the leaderboard to compute feature impact on. Default 5
    metric: str, DR metric to check performance against. Default None. If Default, it will use DR project defined metric
    by_partition: str, whether to use 'validation' or 'crossValidation' partition to get the best model on. Default 'validation'
    feature_list_name: str, name of the feature list to start iterating from. Default None
    ratio: float, ratio of total feature impact that new feature list will contain. Default 0.95
    model_search_params: dict, dictonary of parameters to search the best model. See official DR python api docs. Default None
    use_ranks: Boolean, True to use median rank aggregation or False to use total impact unnormalized. Default True
    
    Returns:
    -----------
    dr.Model object
    """
    
    models = get_best_models(project,
                             metric=metric, 
                             by_partition=by_partition, 
                             start_featurelist_name=feature_list_name,
                             model_search_params = model_search_params)
    
    models = models.values[:n_models]
    
    all_impact = pd.DataFrame()
    
    print("Request Feature Impact calculations")
    # first kick off all FI requests, let DR deal with parallelizing
    for model in models:
        try:
            model.request_feature_impact()
        except:
            pass

    for model in models:
        # This can take some time to compute feature impact
        feature_impact = pd.DataFrame(model.get_or_request_feature_impact(max_wait=60*15))#15min
    
        # Track model name and ID for bookkeeping purposes
        feature_impact['model_type'] = model.model_type
        feature_impact['model_id'] = model.id
        # By sorting and re-indexing, the new index becomes our 'ranking'
        feature_impact = feature_impact.sort_values(by='impactUnnormalized', ascending=False).reset_index(drop=True)
        feature_impact['rank'] = feature_impact.index.values
    
        # Add to our master list of all models' feature ranks
        all_impact = pd.concat([all_impact, feature_impact], ignore_index=True)
     
    #We need to get a threshold number of features to select.
    #based on cumulative sum of impact
    all_impact_agg = all_impact\
            .groupby('featureName')[['impactNormalized','impactUnnormalized']]\
            .sum()\
            .sort_values('impactUnnormalized', ascending=False)\
            .reset_index()
    
    #calculate cumulative feature impact and take first features that possess <ratio> of total impact
    all_impact_agg['impactCumulative'] = all_impact_agg['impactUnnormalized'].cumsum()
    total_impact = all_impact_agg['impactCumulative'].max() * ratio
    tmp_fl = list(set(all_impact_agg[all_impact_agg.impactCumulative <= total_impact]['featureName'].values.tolist()))

    #that will be a number of feature to use
    n_feats = len(tmp_fl)
    
    if use_ranks:
        #get top features based on median rank
        top_ranked_feats = list(all_impact
                                .groupby('featureName')
                                .median()
                                .sort_values('rank')
                                .head(n_feats)
                                .index
                                .values)
    else:
        #otherwise get features based just on total unnormalized feature impact
        top_ranked_feats = list(all_impact_agg.featureName.values[:n_feats])

    ## Create a new featurelist
    featurelist = project.create_modeling_featurelist(f'Reduced FL by Median Rank, top{n_feats}', top_ranked_feats)
    featurelist_id = featurelist.id
    ## Start autopilot
    print('Starting AutoPilot on a reduced feature list')
    project.start_autopilot(featurelist_id=featurelist_id, 
                            prepare_model_for_deployment=True,
                            blend_best_models=False,
                           )
    project.wait_for_autopilot()
    print('... AutoPilot is completed.')
    #return the previous best model
    return models[0]

<b>Define the function to get the best model on the DR leaderboard</b>

We do not want to pick all of the models. <br>
We want to avoid using models trained on a higher (80%,100%) than 3rd stage of autopilot sample size.<br>
We ignore Blenders and Frozen models.
That means we mainly pick models trained on 64% percent of the data.<br>

In [3]:
def get_best_models(project, 
                    metric=None, 
                    by_partition='validation',
                    start_featurelist_name=None,
                    model_search_params=None
                   ):
    '''
    Gets pd.Series of DR model objects sorted by performance. Excludes blenders, frozend and on DR Reduced FL
    
    Parameters:
    -----------
    project: DR project object
    metric: str, metric to use for sorting models on lb, if None, default project metric will be used. Default None
    by_partiton: boolean, whether to use 'validation' or 'crossValidation' partitioning. Default 'validation'
    start_featurelist_name: str, initial featurelist name to get models on. Default None
    model_search_params: dict to pass model search params. Default None
    
    Returns:
    -----------
    pd.Series of dr.Model objects, not blender, not frozen and not on DR Reduced Feature List
    '''
        
    #list of metrics that get better as their value increases
    desc_metric_list = ['AUC', 'Area Under PR Curve', 'Gini Norm', 'Kolmogorov-Smirnov', 'Max MCC', 'Rate@Top5%',
                        'Rate@Top10%', 'Rate@TopTenth%', 'R Squared', 'FVE Gamma', 'FVE Poisson', 'FVE Tweedie',
                        'Accuracy', 'Balanced Accuracy', 'FVE Multinomial', 'FVE Binomial'
                       ]
    
    if not metric:
        metric = project.metric
        if 'Weighted' in metric:
            desc_metric_list = ['Weighted ' + metric for metric in desc_metric_list]
   
    asc_flag = False if metric in desc_metric_list else True
    
    if project.is_datetime_partitioned:
        assert by_partition in ['validation', 'backtesting', 'holdout'], "Please specify correct partitioning, in datetime partitioned projects supported options are: 'validation', 'backtesting', 'holdout' "
        models_df =  pd.DataFrame(
            [[model.metrics[metric]['validation'],
              model.metrics[metric]['backtesting'],
              model.model_category,
              model.is_frozen,
              model.featurelist_name,
              model,
             ] for model in project.get_datetime_models()],
            columns=['validation', 'backtesting', 'category', 'is_frozen', 'featurelist_name', 'model']
        ).sort_values([by_partition], ascending = asc_flag, na_position='last')
    
    else:
        assert by_partition in ['validation', 'crossValidation', 'holdout'], "Please specify correct partitioning, supported options are: 'validation', 'crossValidation', 'holdout' "
        models_df =  pd.DataFrame(
            [[model.metrics[metric]['crossValidation'],
              model.metrics[metric]['validation'],
              model.model_category,
              model.is_frozen,
              model.featurelist_name,
              model,
         ] for model in project.get_models(with_metric = metric, search_params = model_search_params)],
        columns=['crossValidation', 'validation', 'category', 'is_frozen', 'featurelist_name', 'model']
    ).sort_values([by_partition], ascending = asc_flag, na_position='last')
    
    
    if start_featurelist_name:
        return models_df.loc[((models_df.category == 'model')&\
                              (models_df.is_frozen == False)&\
                              (models_df.featurelist_name == start_featurelist_name)
                             ),'model']
    else:
        return models_df.loc[((models_df.category == 'model')&\
                              (models_df.is_frozen == False)&\
                              (models_df.featurelist_name.str.contains('DR Reduced Features M') == False)
                             ),'model']

<b>Let's define the function to perform FIRE several times</b>
<br>This function automatically executes FIRE feature selection algorithm on top N models.
<br>Once the reduced feature list is created, it re-runs the Autopilot and waits until it finishes.
<br>It then automatically sorts the models based on the project metric and computes DR feature impact. And iterates over again.
<br>If the new feature list produces a model worse based on a metric, it will consume one "life". The algorithm will stop performing feature selection when no lifes are available. We start with 3 lifes.

In [4]:
def main_feature_selection(project_id,
                           start_featurelist_name=None,
                           lifes=3,
                           top_n_models=5,
                           partition='validation',
                           main_scoring_metric=None,
                           initial_impact_reduction_ratio=0.95,
                           best_model_search_params=None,
                           use_ranks=True,
                          ):
    '''
    Main function. Meant to get the optimal shortest feature list by repeating feature selection process until stop criteria is met.
    Currently supports Binary, Regression, Multiclass, Datetime partitioned(OTV) and AutoTS DataRobot projects.
    
    Example usage:
    >> import datarobot as dr    
    >> dr.Client(config_path='PATH_TO_DR_CONFIG/drconfig.yaml')
    TIP: set best_model_search_params = {'sample_pct__lte': 65} to avoid using models trained on a higher sample size than 3rd stage of autopilot, which is typically ~64% of the data
    
    >> main_feature_reduction('INSERT_PROJECT_ID',
                              start_featurelist_name=None,
                              lifes=3,
                              top_n_models=5,
                              partition='validation',
                              main_scoring_metric=None,
                              initial_impact_reduction_ratio=0.95,
                              best_model_search_params=None,
                              use_ranks=True)
    
    Parameters:
    -----------
    project_id: str, id of DR project, 
    start_featurelist_name: str, name of feature list to start iterating from. Default None
    lifes: int, stopping criteria, if no best model produced after lifes iterations, stop feature reduction. Default 3
    top_n_models: int, only for 'Rank Aggregation method', get top N best models on the leaderboard. Default 5
    partition: str, whether to use 'validation','crossValidation' or 'backtesting' partition to get the best model on. Default 'validation'
    main_scoring_metric: str, DR metric to check performance against, If None DR project metric will be used
    initial_impact_reduction_ratio: float, ratio of total feature impact that new feature list will contain. Default 0.95
    best_model_search_params: dict, dictonary of parameters to search the best model. See official DR python api docs. Default None
    use_ranks: Boolean, True to use median rank aggregation or False to use total impact unnormalized. Default True
    
    Returns:
    ----------
    dr.Model object of the best model on the leaderboard
    '''
    project = dr.Project.get(project_id)
        
    ratio = initial_impact_reduction_ratio
    assert ratio < 1, "Please specify initial_impact_reduction_ratio < 1"
    
    model_search_params = best_model_search_params
    
    runs = 0
    #main function loop
    while lifes > 0:
        if runs > 0:
            start_featurelist_name = None
        try:    
            best_model = feature_importance_rank_ensembling(project,
                                                               n_models=top_n_models,
                                                               metric=main_scoring_metric,
                                                               by_partition=partition,
                                                               feature_list_name=start_featurelist_name,
                                                               ratio=ratio,
                                                               model_search_params=best_model_search_params,
                                                               use_ranks=use_ranks
                                                              )
        except dr.errors.ClientError as e:
            #decay the ratio
            ratio *= ratio
            print(e, f'\nWill try again with a ratio decay ...  New ratio={ratio:.3f}')
            continue
        
        ##############################
        ##### GET NEW BEST MODEL #####
        ##############################
        
        new_best_model = get_best_models(project,
                                         metric=main_scoring_metric, 
                                         by_partition = partition, 
                                         model_search_params=model_search_params).values[0]

        
        #################################
        ##### PROSESS STOP CRITERIA #####
        #################################
        
        if best_model.id == new_best_model.id:
            #if no better model is produced with a recent run, burn 1 life
            lifes -= 1
            
            #if no lifes left -> stop
            if lifes <= 0:
                print('New model is worse. No lifes left.\nAUTOMATIC FEATURE SELECTION PROCESS HAS BEEN STOPPED')
                return new_best_model
            
            #decay the ratio
            ratio *= ratio
            print(f'New model is worse. One life is burnt.\nRepeat again with decaying the cumulative impact ratio. New ratio={ratio:.3f}')
        
        runs += 1
        print('Run ', runs, ' completed')
    
    return new_best_model

### Create DR project and kick-off AutoPilot

In [5]:
project = dr.Project.create('https://s3.amazonaws.com/datarobot_public_datasets/madelon_combined_80.csv')
project.set_target(target='y',
                   mode=dr.AUTOPILOT_MODE.FULL_AUTO,
                   worker_count=-1,
                  )
project.wait_for_autopilot()
print(project.id)

In progress: 20, queued: 14 (waited: 0s)
In progress: 20, queued: 14 (waited: 3s)
In progress: 20, queued: 14 (waited: 5s)
In progress: 20, queued: 14 (waited: 8s)
In progress: 20, queued: 14 (waited: 11s)
In progress: 20, queued: 14 (waited: 15s)
In progress: 20, queued: 14 (waited: 21s)
In progress: 20, queued: 13 (waited: 30s)
In progress: 20, queued: 13 (waited: 46s)
In progress: 20, queued: 0 (waited: 68s)
In progress: 16, queued: 0 (waited: 90s)
In progress: 11, queued: 0 (waited: 112s)
In progress: 7, queued: 0 (waited: 133s)
In progress: 4, queued: 0 (waited: 155s)
In progress: 2, queued: 0 (waited: 176s)
In progress: 1, queued: 0 (waited: 197s)
In progress: 0, queued: 0 (waited: 219s)
In progress: 8, queued: 0 (waited: 240s)
In progress: 8, queued: 0 (waited: 261s)
In progress: 8, queued: 0 (waited: 282s)
In progress: 7, queued: 0 (waited: 304s)
In progress: 7, queued: 0 (waited: 325s)
In progress: 5, queued: 0 (waited: 347s)
In progress: 3, queued: 0 (waited: 368s)
In progres

Once Project's inital AutoPilot is finished, let's start performing feature selection.<br>

We start new Autopilot on feature list based on median rank aggregation of feature impacts across top 5 models trained on "Informative Features" list.<br>

In [6]:
#feel free adjust function's parameters for your purposes. 
best_model = main_feature_selection(project.id,
                                    partition='crossValidation',
                                    best_model_search_params={'sample_pct__lte': 65})

Request Feature Impact calculations
Starting AutoPilot on a reduced feature list
In progress: 0, queued: 0 (waited: 0s)
In progress: 0, queued: 0 (waited: 1s)
In progress: 0, queued: 0 (waited: 3s)
In progress: 0, queued: 0 (waited: 4s)
In progress: 0, queued: 0 (waited: 6s)
In progress: 0, queued: 0 (waited: 9s)
In progress: 0, queued: 0 (waited: 13s)
In progress: 0, queued: 0 (waited: 21s)
In progress: 20, queued: 14 (waited: 35s)
In progress: 20, queued: 14 (waited: 57s)
In progress: 20, queued: 13 (waited: 80s)
In progress: 20, queued: 4 (waited: 102s)
In progress: 16, queued: 0 (waited: 124s)
In progress: 11, queued: 0 (waited: 146s)
In progress: 7, queued: 0 (waited: 168s)
In progress: 4, queued: 0 (waited: 189s)
In progress: 2, queued: 0 (waited: 210s)
In progress: 0, queued: 0 (waited: 232s)
In progress: 8, queued: 0 (waited: 253s)
In progress: 8, queued: 0 (waited: 274s)
In progress: 8, queued: 0 (waited: 296s)
In progress: 7, queued: 0 (waited: 317s)
In progress: 5, queued: 0

In [7]:
print(f"The best model has {project.metric} score = {best_model.metrics[project.metric]['crossValidation']} on the cross-validation partition \
on the list of {len(best_model.get_features_used())} features")

The best model has LogLoss score = 0.26109799999999994 on the cross-validation partition on the list of 19 features
